## Image Data Preparation

The data processing is done through the LoadImg (modules/LoadImg.py) module. The images dataset is first split into train and validation datasets. Then, their size and color mode are changed (using keras.preprocessing module).

In [ ]:
from modules import LoadImg

## load data
# The color_mode is either 'rgb' or 'grayscale' (default).
X_train, X_test, y_train, y_test, cls_indices = LoadImg.Dataset.prep_datasets(
    ver_ratio=0.2, container_path='data/ImageEveryUnit',
    final_img_width=79, final_img_height=79 * 2,
    color_mode="grayscale", random_state=1911)


A Covolutional Neural Network (CNN) model is trained and used for predicting a gauge images categories. There are 30 categories that show the pressure level collected by the gauge and one NaN categorie (31 categories all together).

The CNN model consists of three CNN and two Fully Connected (FC) layers (the layers are defined in the NewLayers module, _modules/NewLayers.py_, and are combined in the NeuralNet module, _modules/NeuralNet.py_). The activation function of all layers is Rectified Linear Unit (relu) except the last layer which is softmax.

During the training, the cross-entropy function and Adam algorithm are employed as the cost function and optimizer, respectively. The model performance is evaluated via the accuracy of the validation dataset and the model checkpoint is saved (in the _logs/models/main_ folder) whenever the accuracy is improved. The models that have high validation accuracy for each specific category are also saved (in folders with the same name as the category name like _logs/models/psi_\__2_); these models can be later used for the final ensemble model construction.

First, a model was trained using images with the same width/height ratio as the original images, which is one here. The hyperparameters of this model were manually tunned and the ones that resulted relatively better model performance were used for the subsequent models. Having trained the initial model, the width/height ratios of the images were changed and for each ratio a model was trained and its saved checkpoint was manually copy-pasted to the _logs/historical_\__models_ for the final ensemble model.

In [ ]:
from modules import OptimizeAndLog
import importlib

##
importlib.reload(OptimizeAndLog)
OptimizeAndLog.OptimizerLogger. \
    train_and_save_logs_three_CNN(filter_size1=17, num_filters1=45, strides_1=[1, 7, 7, 1],
                                  use_pooling_1=True, pooling_ksize_1=[1, 4, 4, 1], pooling_strides_1=[1, 4, 4, 1],
                                  ##
                                  filter_size2=7, num_filters2=17, strides_2=[1, 5, 5, 1],
                                  use_pooling_2=True, pooling_ksize_2=[1, 3, 3, 1], pooling_strides_2=[1, 3, 3, 1],
                                  ##
                                  filter_size3=1, num_filters3=7, strides_3=[1, 1, 1, 1],
                                  use_pooling_3=False, pooling_ksize_3=None, pooling_strides_3=None,
                                  ##
                                  fc_size=86,
                                  num_iterations=2,
                                  learning_rate=2e-4, momentum=None,
                                  X_train=X_train, y_train=y_train,
                                  X_test=X_test, y_test=y_test,
                                  models_log_path='./logs/models/',
                                  cls_indices=cls_indices,
                                  padding='SAME',
                                  device_name="/cpu:0")